In [1]:
from lsa import *
import matplotlib.pyplot as plt
import csv
import os
import json
import time

In [2]:
data_dir = 'data'

In [3]:
data_files = [os.path.join(data_dir, f) for f in DATA_FILES]

In [4]:
df_data = load_data(data_files)

In [5]:
df_data

,title,author,url,content,id,publication
0,It’s Possible to Hack a Phone With Sound Waves...,John Markoff,NaN,SAN FRANCISCO — A security loophole that wo...,19284,New York Times
1,Politically Correct School Officials Reject ’G...,Tom Ciccotta,NaN,A report from The Baltimore Sun claims that Ba...,44077,Breitbart
2,"Clinton, Sanders use N.H. primary to frame lon...",Philip Rucker,https://web.archive.org/web/20160206013008/htt...,"CONCORD, N. H. — For the Democratic presi...",209714,Washington Post
3,The ambitious Democrat who can make Trump Univ...,Charles Gasparino,http://nypost.com/2016/03/08/the-ambitious-dem...,Add one more bizarre twist to the 2016 preside...,135930,New York Post
4,People Are Slamming Tribeca Film Festival For...,Susan Cheng,https://web.archive.org/web/20160325115729/htt...,Organizers for the upcoming 2016 Tribeca Film...,106460,Buzzfeed News
...,...,...,...,...,...,...
995,Autopsy shows Keith Lamont Scott had four guns...,Wesley Lowery,https://web.archive.org/web/20161013005231/htt...,Keith Lamont Scott was struck at least th...,215980,Washington Post
996,Scientists found bacteria inside rocks — here’...,The Cosmic Companion,https://thenextweb.com/syndication/2020/04/17/...,The discovery of bacteria inside rocks under t...,73470,The Next Web
997,"Little Richard, rock 'n' roll pioneer, has die...",NaN,https://www.cbsnews.com/news/little-richard-ro...,"Little Richard, one of the chief architects of...",73471,CBS NEWS
998,Silicon Valley’s favorite magician reimagines ...,Zachary Crockett,https://thehustle.co/silicon-valleys-favorite-...,"Daniel Chan has stumped billionaires, CEOs, a...",73472,The Hustle


In [25]:
# Cas preprocesingu stemming vs lemmatizace

In [26]:
start = time.time()
df_words_stemmed = preprocess_docs(df_data, use_lemmatizer=False, remove_numbers=True)
end = time.time()

print("Čas preprocesingu s použitím stemmingu: " + str(end-start) + " sekund")

Čas preprocesingu s použitím stemmingu: 11.415580749511719 sekund


In [ ]:
start = time.time()
df_words_lemmatized = preprocess_docs(df_data, use_lemmatizer=True, remove_numbers=True)
end = time.time()

print("Čas preprocesingu s použitím lemmatizace: " + str(end-start) + " sekund")

In [ ]:
# Graf singularnich hodnot stemming versus lemmatizace

In [ ]:
df_frequency_stemmed = get_term_by_document_frequency(df_words_stemmed)
df_frequency_lemmatized = get_term_by_document_frequency(df_words_lemmatized)
df_reduced_stemmed = reduce_terms(df_frequency_stemmed, max_df=1, min_df=1, max_terms=0,
                          keep_less_freq=False)
df_reduced_lemmatized = reduce_terms(df_frequency_lemmatized, max_df=1, min_df=1, max_terms=0,
                          keep_less_freq=False)

df_tf_idf_stemmed = get_tf_idf(df_reduced_stemmed)
df_tf_idf_lemmatized = get_tf_idf(df_reduced_lemmatized)

values_stemmed = df_tf_idf_stemmed.fillna(0).to_numpy()
values_lemmatized = df_tf_idf_lemmatized.fillna(0).to_numpy()

U, s_eigen_stemmed, V = np.linalg.svd(values_stemmed, full_matrices=False)
U, s_eigen_lemmatized, V = np.linalg.svd(values_lemmatized, full_matrices=False)

s_eigen_norm_stemmed = [x / s_eigen_stemmed[0] for x in s_eigen_stemmed]
s_eigen_norm_lemmatized = [x / s_eigen_lemmatized[0] for x in s_eigen_lemmatized]

In [ ]:
df_reduced_stemmed

In [ ]:
df_reduced_lemmatized

In [ ]:
fig = plt.figure()
plt.plot(s_eigen_norm_stemmed)
plt.plot(s_eigen_norm_lemmatized)
plt.xlabel('Singulárních hodnoty')
plt.ylabel('Poměr vůči nejvyšší singulární hodnotě')
#fig.savefig('../report/images/singular_values.png')
plt.show()

In [ ]:
#Graf singularnich hodnot

In [6]:
df_words = preprocess_docs(df_data, use_lemmatizer=True, remove_numbers=True)
df_frequency = get_term_by_document_frequency(df_words)
df_reduced = reduce_terms(df_frequency, max_df=1, min_df=1, max_terms=0,
                          keep_less_freq=False)
df_tf_idf = get_tf_idf(df_reduced)

In [ ]:
df_reduced

In [ ]:
values = df_tf_idf.fillna(0).to_numpy()

In [ ]:
U, s_eigen, V = np.linalg.svd(values, full_matrices=False)

In [ ]:
s_eigen

In [ ]:
s_eigen[0]

In [ ]:
s_eigen_norm = [x / s_eigen[0] for x in s_eigen]

In [ ]:
s_eigen_norm

In [ ]:
fig = plt.figure()
plt.plot(s_eigen_norm)
plt.xlabel('Singulárních hodnoty')
plt.ylabel('Poměr vůči nejvyšší singulární hodnotě')
fig.savefig('../report/images/singular_values.png')
plt.show()

In [ ]:
# Cas vykonani dotazu pri sekvencnim pruchodu vs LSI pruchodu databazi

In [7]:
df_concept_by_doc, df_query_projection = transform_to_concept_space(df_tf_idf, k=0, customSVD=False)

start = time.time()
best_match = get_n_nearest(df_tf_idf,df_concept_by_doc, df_query_projection, 999, n=10)
end = time.time()
time_seq = end - start
print("Čas vykonání dotazu při sekvenčním průchodu databází: " + str(time_seq) + " sekund")

Čas vykonání dotazu při sekvenčním průchodu databází: 0.17969512939453125 sekund


In [8]:
df_concept_by_doc, df_query_projection = transform_to_concept_space(df_tf_idf, k=200, customSVD=False)

start = time.time()
best_match = get_n_nearest(df_tf_idf,df_concept_by_doc, df_query_projection, 999, n=10)
end = time.time()
time_norm = end - start
print("Čas vykonání dotazu při průchodu databází pomocí LSI vektorového modelu: " + str(time_norm) + " sekund")

Čas vykonání dotazu při průchodu databází pomocí LSI vektorového modelu: 0.11586117744445801 sekund


In [9]:
print("Rozdíl: " + str(time_seq - time_norm))

Rozdíl: 0.06383395195007324
